In [140]:
import numpy as np
import pandas as pd
import os

In [141]:
registered_business_path = os.path.join('Cleaned Datasets', 'cleaned_registered_food_services.csv')
health_scores_path = os.path.join('Cleaned Datasets', 'health_scores.csv')

geocoded_registered_business_path = os.path.join('Cleaned Datasets', 'geocoded_addresses-2018.csv')
geocoded_health_scores_path = os.path.join('Cleaned Datasets', 'geocoded_health_scores_addresses-2018.csv')


In [144]:
registered_business_df = pd.read_csv(registered_business_path)
health_scores_df = pd.read_csv(health_scores_path)

geocoded_rb_df = pd.read_csv(geocoded_registered_business_path)
geocoded_hs_df = pd.read_csv(geocoded_health_scores_path)
geocoded_hs_df = geocoded_hs_df.drop_duplicates(subset=['google_place_id'], keep='first').reset_index(drop=True)

In [145]:
geocoded_hs_df.head()

,accuracy,formatted_address,google_place_id,input_string,latitude,longitude,number_of_results,postcode,status,type
0,ROOFTOP,"1200 Van Ness Ave 3RD FLOOR, San Francisco, CA...",EjozUkQgRkxPT1IsIDEyMDAgVmFuIE5lc3MgQXZlLCBTYW...,"1200 VAN NESS AVE, 3RD FLOOR",37.786777,-122.421335,1,94109.0,OK,subpremise
1,ROOFTOP,"500 California St 2ND FLOOR, San Francisco, C...",EjsyTkQgIEZMT09SLCA1MDAgQ2FsaWZvcm5pYSBTdCwgU2...,"500 CALIFORNIA ST, 2ND FLOOR",37.793123,-122.403101,1,94104.0,OK,subpremise
2,ROOFTOP,"2801 Leavenworth St, San Francisco, CA 94133, USA",ChIJu2t7buGAhYARabsFEzVXIqw,2801 LEAVENWORTH ST,37.807021,-122.419384,2,94133.0,OK,premise
3,ROOFTOP,"3202 Folsom St, San Francisco, CA 94110, USA",ChIJARWKcUV-j4ARfAqXO018-80,3202 FOLSOM ST,37.747104,-122.413831,2,94110.0,OK,premise
4,ROOFTOP,"747 Irving St, San Francisco, CA 94122, USA",ChIJE9-qSVyHhYARqRpbrJ9rHlA,747 IRVING ST,37.763933,-122.466047,2,94122.0,OK,street_address


In [147]:
#Merge the registered business df with the geocodes to get unique location id
merge_registered_business_df = registered_business_df.merge(geocoded_rb_df, left_on='street_address', right_on='input_string')
len(merge_registered_business_df)
merge_registered_business_df.columns.values

array(['location_id', 'ownership_name', 'business_name', 'category',
       'lic_code', 'street_address', 'State', 'City', 'zipcode',
       'business_start_date', 'business_end_date', 'neighborhoods',
       'business_location', 'supervisor_district', 'Lifespan', 'accuracy',
       'formatted_address', 'google_place_id', 'input_string', 'latitude',
       'longitude', 'number_of_results', 'postcode', 'status', 'type'],
      dtype=object)

In [149]:
#Get columns we need
merge_registered_business_df = merge_registered_business_df[['location_id', 'ownership_name', 'business_name', 'category',
                            'lic_code', 'street_address', 'State', 'City', 'zipcode', 
                            'business_start_date', 'business_end_date', 'neighborhoods', 'business_location',
                            'supervisor_district', 'Lifespan', 'formatted_address', 'google_place_id', 'latitude', 'longitude']]

In [152]:
#Merge the healthscores  df with the geocodes to get unique location id
merge_health_scores_df = health_scores_df.merge(gecoded_hs_df, left_on='address', right_on='input_string')
len(merge_health_scores_df)

4025

In [153]:
merge_health_scores_df.head()
merge_health_scores_df.columns.values

array(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude_x', 'longitude_x', 'phone_number', 'score', 'date',
       'type_x', 'year', 'month', 'Unnamed: 0', 'accuracy',
       'formatted_address', 'google_place_id', 'input_string',
       'latitude_y', 'longitude_y', 'number_of_results', 'postcode',
       'status', 'type_y'], dtype=object)

In [154]:
#get the columns we need
merge_health_scores_df = merge_health_scores_df[['business_id', 'name', 'score', 'year', 'formatted_address']]
merge_health_scores_df.head()

,business_id,name,score,year,formatted_address
0,19,NRGIZE LIFESTYLE CAFE,94.0,2017.0,"1200 Van Ness Ave 3RD FLOOR, San Francisco, CA..."
1,19,NRGIZE LIFESTYLE CAFE,96.0,2018.0,"1200 Van Ness Ave 3RD FLOOR, San Francisco, CA..."
2,19,NRGIZE LIFESTYLE CAFE,94.0,2016.0,"1200 Van Ness Ave 3RD FLOOR, San Francisco, CA..."
3,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,96.0,2016.0,"500 California St 2ND FLOOR, San Francisco, C..."
4,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,98.0,2016.0,"500 California St 2ND FLOOR, San Francisco, C..."


In [155]:
#Merge the two files to see if there are any matches
merged_df = merge_health_scores_df.merge(merge_registered_business_df, left_on='formatted_address', right_on='formatted_address')
len(merged_df)
# merged_df.head(20)

3021

In [156]:
merged_df.head(30)

,business_id,name,score,year,formatted_address,location_id,ownership_name,business_name,category,lic_code,...,zipcode,business_start_date,business_end_date,neighborhoods,business_location,supervisor_district,Lifespan,google_place_id,latitude,longitude
0,48,ART'S CAFE,94.0,2016.0,"747 Irving St, San Francisco, CA 94122, USA",0318022-01-001,Yoon Hae Ryong,Arts Cafe,Food Services,"Restaurant Less Than 1,000 Sqft",...,94122.0,07/01/1998,NaN,Inner Sunset,"747 IRVING ST\nSan Francisco, CA 94122\n(37.76...",5.0,NaN,ChIJE9-qSVyHhYARqRpbrJ9rHlA,37.763933,-122.466047
1,48,ART'S CAFE,94.0,2018.0,"747 Irving St, San Francisco, CA 94122, USA",0318022-01-001,Yoon Hae Ryong,Arts Cafe,Food Services,"Restaurant Less Than 1,000 Sqft",...,94122.0,07/01/1998,NaN,Inner Sunset,"747 IRVING ST\nSan Francisco, CA 94122\n(37.76...",5.0,NaN,ChIJE9-qSVyHhYARqRpbrJ9rHlA,37.763933,-122.466047
2,61,CHOWDERS,94.0,2016.0,"Beach St & The Embarcadero, San Francisco, CA ...",1182485-03-181,Hayes Brian D & Maureen A,Crabcakes And Sweets,Food Services,NaN,...,94133.0,11/15/1981,NaN,NaN,NaN,NaN,NaN,ChIJHSGzi_yAhYARnrPmDWAx9ro,37.808673,-122.409821
3,61,CHOWDERS,94.0,2016.0,"Beach St & The Embarcadero, San Francisco, CA ...",1183392-03-181,Hayes Brian D & Maureen A,The Churro Factory,Food Services,Take-Out Establishment,...,94133.0,11/15/1981,NaN,NaN,NaN,NaN,NaN,ChIJHSGzi_yAhYARnrPmDWAx9ro,37.808673,-122.409821
4,61,CHOWDERS,94.0,2016.0,"Beach St & The Embarcadero, San Francisco, CA ...",0027895-01-001,Swiss Louis Restaurant Inc,Swiss Louis Restaurant Inc,Food Services,"Restaurant Greater Than 2,000 Sqft",...,94133.0,10/01/1968,NaN,NaN,"204 San\nFrancisco, CA 94133\n",NaN,NaN,ChIJHSGzi_yAhYARnrPmDWAx9ro,37.808673,-122.409821
5,61,CHOWDERS,94.0,2016.0,"Beach St & The Embarcadero, San Francisco, CA ...",0327947-01-001,Bubba Gump Shrimp Restaurants,"Bubba Gump Shrimp Co.,rest Mkt",Food Services,NaN,...,94133.0,12/31/2010,NaN,NaN,"M210 San\nFrancisco, CA 94133\n",NaN,NaN,ChIJHSGzi_yAhYARnrPmDWAx9ro,37.808673,-122.409821
6,61,CHOWDERS,94.0,2016.0,"Beach St & The Embarcadero, San Francisco, CA ...",0371382-03-001,Premier Desserts Inc,Blue Chip Cookies Pier 39,Food Services,NaN,...,94133.0,10/01/2003,NaN,NaN,"6 San\nFrancisco, CA 94133\n(37.799393, -122.4...",NaN,NaN,ChIJHSGzi_yAhYARnrPmDWAx9ro,37.808673,-122.409821
7,61,CHOWDERS,94.0,2017.0,"Beach St & The Embarcadero, San Francisco, CA ...",1182485-03-181,Hayes Brian D & Maureen A,Crabcakes And Sweets,Food Services,NaN,...,94133.0,11/15/1981,NaN,NaN,NaN,NaN,NaN,ChIJHSGzi_yAhYARnrPmDWAx9ro,37.808673,-122.409821
8,61,CHOWDERS,94.0,2017.0,"Beach St & The Embarcadero, San Francisco, CA ...",1183392-03-181,Hayes Brian D & Maureen A,The Churro Factory,Food Services,Take-Out Establishment,...,94133.0,11/15/1981,NaN,NaN,NaN,NaN,NaN,ChIJHSGzi_yAhYARnrPmDWAx9ro,37.808673,-122.409821
9,61,CHOWDERS,94.0,2017.0,"Beach St & The Embarcadero, San Francisco, CA ...",0027895-01-001,Swiss Louis Restaurant Inc,Swiss Louis Restaurant Inc,Food Services,"Restaurant Greater Than 2,000 Sqft",...,94133.0,10/01/1968,NaN,NaN,"204 San\nFrancisco, CA 94133\n",NaN,NaN,ChIJHSGzi_yAhYARnrPmDWAx9ro,37.808673,-122.409821


In [157]:
path = os.path.join('Cleaned Datasets', 'final-merged.csv')
merged_df.to_csv(path, index=False)